In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import random
import time

class EVCSIndividual:

    '''
    individual of demand oriented method
    '''
    def __init__(self,data,ev_ct,ec_cp,ev_n):

        self.data=data  #候选数据
        self.evcs=np.zeros(ev_n)  #每个充电站的车数
        self.individual=[]
        self.fitness=[]
        self.ev_ct=ev_ct
        self.ev_cp=ec_cp
        self.ev_n=ev_n

    def GenerateIndividual(self):
        '''
        generate a random chromsome for genetic algorithm
        '''
        gene = list()    #问题的解，基因，种群中的个体：[0，...，city_size]
        for i in range(len(self.data)):
            ind=random.choices(self.data[i])
            copy_ind=copy.deepcopy(ind[0])
            copy_ind.append(i)
            gene.append(copy_ind)
        random.shuffle(gene)
        self.individual=gene

    def calculateFitness(self,cd_n=5):
        '''
        calculate the fitness of the indiviual
        '''
        ev_n=np.zeros((self.ev_n,cd_n))  #单位时间内每个电桩充电车数
        cd_wt=np.zeros((self.ev_n,cd_n)) #充电桩工作总时间
        cd_st=np.zeros((self.ev_n,cd_n)) #充电桩空闲时间
        st=0.   #电桩空闲总时间  
        qt=0.   #电车排队总时间 
        tt=0.   #充电总时间 到充电桩时间+排队时间+充电时间     
        cost=0. #充电总成本成本
        n=[]  
        x_n=0
        for i in range(len(self.individual)): #len(self.indiviual)
            cost+=self.individual[i][1]+self.ev_cp[self.individual[i][0]]*self.ev_ct[self.individual[i][2]]
            k=int(self.evcs[self.individual[i][0]]%cd_n)
            self.evcs[self.individual[i][0]]+=1
            if cd_wt[self.individual[i][0]][k]<60:
                ev_n[self.individual[i][0]][k]+=1
            if self.individual[i][1]<cd_wt[self.individual[i][0]][k]:
                tt+=cd_wt[self.individual[i][0]][k]+self.ev_ct[self.individual[i][0]]
                qt+=cd_wt[self.individual[i][0]][k]-self.individual[i][1]
                cd_wt[self.individual[i][0]][k]=cd_wt[self.individual[i][0]][k]+self.ev_ct[self.individual[i][0]]
                x_n+=1
            else:
                tt+=self.individual[i][1]+self.ev_ct[self.individual[i][0]]
                st+=self.individual[i][1]-cd_wt[self.individual[i][0]][k]
                cd_st[self.individual[i][0]][k]+=self.individual[i][1]-cd_wt[self.individual[i][0]][k]
                cd_wt[self.individual[i][0]][k]=self.individual[i][1]+self.ev_ct[self.individual[i][0]]
        revnue=0.
        t_ev_n=0
        t_st_v=0
        for i in range(self.ev_n):
            for j in range(cd_n):
                revnue+=(cd_wt[i][j]-cd_st[i][j])*self.ev_cp[i]
                t_ev_n+=ev_n[i][j]
                t_st_v+=cd_st[i][j]
        
        n.append(revnue)      #总收益  
        n.append(cost)        #总成本
        n.append(tt)          #总时间
        n.append(qt)          #总排队时间
        n.append(st)          #总空闲时间
        n.append(t_ev_n)      #单位时间每个充电桩充电电车总数
        self.fitness=np.array(n)

class GeneticAlgorithm:

    def __init__(self, t_c_l, ev_ct, ev_cp, n,c_rate=0.7, m_rate=0.3, pop_size=200, maxnum=200):
        self.pop_size = pop_size#50
        self.fitness = np.zeros(self.pop_size)#Pop_size 为50，种群个数，应该修改为参数，可传递任意种群个数
        self.c_rate = c_rate #交叉率，0.7，修改为可变参数
        self.m_rate = m_rate #变异率，0.05，修改为
        self.maxiternum = maxnum# 最大迭代次数
        self.population=[]
        self.bestfitness=0.   #最好的适应值
        self.besttruefitness=[] # 最好的真适应值
        self.bestIndex=0
        self.bestgene=np.array([])
        #self.dw = Draw()    #绘图类
        self.trace = np.zeros((self.maxiternum, 2))
        self.avefitness=0.
        self.data=t_c_l
        self.ev_ct=ev_ct
        self.ev_cp=ev_cp
        self.cs_n=n
        self.bestindividual=[]   #最好的个体
    
    def initialize(self):
        '''
        initialize the population of GA
        '''
        for i in range(0, int(self.pop_size)):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            self.population.append(ind)    
        
    def evaluation(self):
        '''
        evaluation the fitness of the population
        '''
        for i in range(0, int(self.pop_size)):
            self.population[i].calculateFitness()
            self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
            
    def selection(self):
        for i in range(self.pop_size):
            if i != self.bestIndex and self.fitness[i] > self.avefitness:
                pi = self.cross(self.population[self.bestIndex].individual, self.population[i].individual)
                self.population[i].individual= self.mutate(pi)
                self.population[i].individual=sorted(self.population[i].individual, key=(lambda x: [x[0],x[1]]))
                self.population[i].calculateFitness()
                self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
    
    def crossoverMutation(self):
         for j in range(self.pop_size):
                r = np.random.randint(0, self.pop_size - 1)
                if j != r:
                    nind = self.cross(self.population[j].individual, self.population[r].individual)    #交叉种群中第j,r个体的基因
                    self.population[j].individual = self.mutate(nind)    #突变种群中第j个体的基因
                    self.population[j].calculateFitness()
                    self.fitness[j] = 0.3*(self.population[j].fitness[3]/40000)+0.4*(self.population[j].fitness[4]/3000)+0.3*(1-self.population[j].fitness[5]/899)
       
    def cross(self, parent1, parent2):
        """crossover"""
        if np.random.rand() > self.c_rate:
            return parent1
        index1 = np.random.randint(0, len(parent1) - 1)
        index2 = np.random.randint(index1,  len(parent1)- 1)
        tempGene = parent2[index1:index2]  # 交叉的基因片段
        tempGene_t = pd.DataFrame(tempGene,columns=["0","1","2"])
        tempGene_c = list(tempGene_t["2"])
        newGene = []
        p1len = 0
        for g in parent1:
            if p1len == index1:
                newGene.extend(tempGene)  # 插入基因片段
            if g[2] not in tempGene_c:
                newGene.append(g)
            p1len += 1
        
        if len(newGene)!=len(parent1):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            return ind.individual
        
        return newGene
    
    def mutate(self,gene):
        """mutation"""
        if np.random.rand() > self.m_rate:
            return gene
        index1 = np.random.randint(0, len(gene) - 1)
        index2 = np.random.randint(index1, len(gene) - 1)
        newGene = self.reverse_gen(gene, index1, index2)
        if len(newGene)!=len(gene):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            return ind.individual
        return newGene
    
    def reverse_gen(self, gen, i, j):
        #函数：翻转基因中i到j之间的基因片段
        if i >= j:
            return gen
        if j > len(gen) - 1:
            return gen
        parent1 = copy.deepcopy(gen)
        tempGene = parent1[i:j]
        parent1[i:j]=tempGene[::-1]
        return parent1
    
    def solve(self):
        self.t = 0
        self.initialize()
        self.evaluation()
        self.bestfitness = np.min(self.fitness)
        self.bestIndex = np.argmin(self.fitness)
        self.bestgene = copy.deepcopy(self.population[self.bestIndex])
        self.besttruefitness=self.population[self.bestIndex].fitness
        self.bestindividual=self.population[self.bestIndex].individual
        self.avefitness = np.mean(self.fitness)
        self.trace[self.t, 0] = self.fitness[self.bestIndex]
        self.trace[self.t, 1] = self.avefitness
        print("Iteration:%d, fitness:%f, average fitness:%f, best individual:%s." % (
            self.t, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))

        while self.t < self.maxiternum - 1:
            self.t += 1
            self.selection()
            self.crossoverMutation()
            localbest = np.min(self.fitness)
            self.bestIndex = np.argmin(self.fitness)
            if localbest<self.bestfitness:
                self.bestfitness=localbest
                self.besttruefitness=self.population[self.bestIndex].fitness
                self.bestindividual=self.population[self.bestIndex].individual
            self.bestgene = copy.deepcopy(self.population[self.bestIndex])
            self.avefitness = np.mean(self.fitness)            
            self.trace[self.t, 0] = self.fitness[self.bestIndex]
            self.trace[self.t, 1] = self.avefitness
            print("Iteration:%d, fitness:%f, average fitness:%f, best individual:%s." % (
                self.t, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))
        print("True Best Fitness:",self.besttruefitness)
        print("Best Fitness:",self.bestfitness)
        print("Average of queuing time (minute):",self.besttruefitness[3]/899)
        print("Average of idle time (minute):",self.besttruefitness[4]/170)
        print("Number of charged EV within an hour:",int(self.besttruefitness[5]))
        print("Total cost of all EV:",self.besttruefitness[1])
        print("Total revenue of EVCS:",self.besttruefitness[0])
        print(list(self.trace[:,0]))
        
def main():
    '''access dataset'''
    data = pd.read_csv("chargingstations.csv", delimiter=";", header=None).values  #读取34省市数据，以';'分割，dataframe
    cities = data[:, 1:]# 0行至最后一行，1列至最后一列
    cities_name = data[:, 0] #0行至最后一行，第0列
    city_size = data.shape[0]# shape属性返回包含行和列的元组
    locations = np.arange(cities.shape[0])    #问题的解，基因，种群中的个体：[0，...，city_size]

    ev_data_1000= pd.read_csv('ev_data_1000.csv').values 
    ev_x=ev_data_1000[:, 1:2]
    ev_x=[j for i in ev_x for j in i]
    ev_y=ev_data_1000[:, 2:3]
    ev_y=[j for i in ev_y for j in i]
    ev_ld=ev_data_1000[:, 3:4]
    ev_ld=[j for i in ev_ld for j in i]
    ev_ct=ev_data_1000[:, 4:5]
    ev_ct=[j for i in ev_ct for j in i]
    ev_cp_34= pd.read_csv('ev_cp_34.csv').values 
    ev_cp=ev_cp_34[:,1:]
    ev_cp=[j for i in ev_cp for j in i]
    
    '''charging stations'''
    t_c_l=[] #每个电车的候选充电站
    for i in range(1000):
        c_l=[]
        for j in range(34):
            d=np.sqrt((ev_x[i] - cities[j][0]) ** 2 + (ev_y[i] - cities[j][1]) ** 2)
            #d=ct_distance(ev_x[i],ev_y[i],cities[j])
            if d<ev_ld[i]:
                c=[]
                c.append(int(j))      # 充电站序号
                c.append(d)      # 距离充电站距离，也是行驶时间
                c_l.append(c)
        t_c_l.append(c_l)        

    for i in range(len(t_c_l)-1,-1,-1):
        if t_c_l[i]==[]:
            del ev_x[i]
            del ev_y[i]
            del ev_ld[i]
            del ev_ct[i]
            del t_c_l[i]
    GA=GeneticAlgorithm(t_c_l,ev_ct,ev_cp,len(cities))
    GA.solve()
    
        
#目前最优[ 16575.94745795 175006.78616601  55609.61504463  30785.9283487   2206.67604534    634.        ]
#适应值:0.594212,平均适应值:0.659067,最优结果:[ 16770.97487788 171025.98243471  55252.89521337  30548.93880873  2089.99381702    640.        ]
#适应值:0.508712,平均适应值:0.521612,最优结果:[ 16763.10019171 170352.63143501  53207.77215977  28552.77795326  1608.58002773    659.        ]
#适应值:0.507338,平均适应值:0.526123,最优结果:[ 16759.53057865 174122.93111212  52971.02862662  28044.18074583  1671.93032886    677.        ]
#适应值:0.318232,平均适应值:0.318232,最优结果:[ 16665.83843136 175328.5014536   47437.33551609  22497.46944712  675.75942351    721. ]       ]
#适应值:0.315828,平均适应值:0.378350,最优结果:[ 16533.31368097 174995.88255584  46663.01202116  21854.45136753  741.45495666    740.   ]     ]
#适应值:0.307763,平均适应值:0.376329,最优结果:[ 16589.821804   171644.80267732  46448.66209076  21793.67946582  664.36665068    732.        ]
if __name__ == '__main__':
    start=time.time()
    main()
    end=time.time()
    print("Time consuming:",end-start)  

Iteration:0, fitness:0.618667, average fitness:0.656702, best individual:[16559.31420929 28414.74120048 56265.01212732 31482.94780829
  2188.32778708   627.        ].
Iteration:1, fitness:0.327826, average fitness:0.543776, best individual:[16657.25206822 28425.10899786 48237.12751697 23446.43552146
   679.31947174   715.        ].
Iteration:2, fitness:0.323521, average fitness:0.420051, best individual:[16600.20291007 28345.65702516 47838.19508162 23260.83026302
   687.50816843   727.        ].
Iteration:3, fitness:0.321738, average fitness:0.411160, best individual:[16660.31273682 28389.4707432  47788.31827142 23118.07014548
   672.15596659   723.        ].
Iteration:4, fitness:0.321738, average fitness:0.412131, best individual:[16660.31273682 28389.4707432  47788.31827142 23118.07014548
   672.15596659   723.        ].
Iteration:5, fitness:0.320070, average fitness:0.393978, best individual:[16644.7109393  28369.21102364 47588.97105694 22940.18710812
   672.15596659   724.        ]

Iteration:49, fitness:0.310389, average fitness:0.362038, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:50, fitness:0.310389, average fitness:0.375865, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:51, fitness:0.310389, average fitness:0.384501, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:52, fitness:0.310389, average fitness:0.355864, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:53, fitness:0.310389, average fitness:0.361453, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:54, fitness:0.310389, average fitness:0.372765, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.   

Iteration:98, fitness:0.310389, average fitness:0.361966, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:99, fitness:0.310389, average fitness:0.364296, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:100, fitness:0.310389, average fitness:0.364810, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:101, fitness:0.310389, average fitness:0.367792, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:102, fitness:0.310389, average fitness:0.372338, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:103, fitness:0.310389, average fitness:0.359397, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730

Iteration:147, fitness:0.310389, average fitness:0.363473, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:148, fitness:0.310389, average fitness:0.361140, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:149, fitness:0.310389, average fitness:0.366117, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:150, fitness:0.310389, average fitness:0.375612, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:151, fitness:0.310389, average fitness:0.372836, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:152, fitness:0.310389, average fitness:0.364719, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   7

Iteration:196, fitness:0.310389, average fitness:0.384255, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:197, fitness:0.310389, average fitness:0.378193, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:198, fitness:0.310389, average fitness:0.356888, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
Iteration:199, fitness:0.310389, average fitness:0.362193, best individual:[16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ].
True Best Fitness: [16579.55776793 28382.24712638 46755.92288671 22072.14107251
   663.39209573   730.        ]
Best Fitness: 0.31038933302469646
Average of queuing time (minute): 24.551881059524806
Average of idle time (minute): 3.9023064454517065
Number of charged EV within an hour: 730
Total cost of all EV: 28382.247126